<a href="https://colab.research.google.com/github/Meenasree16/meena/blob/main/qa_score_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup & Sample Data

In [8]:
# Sample transcript (load from uploaded file or paste)
import json

# Upload the transcript JSON file
from google.colab import files
uploaded = files.upload()

# Load and parse the transcript file
filename = list(uploaded.keys())[0]
with open(filename, 'r') as f:
    transcript = json.load(f)

# Combine all Executive utterances
executive_transcript = " ".join([
    turn["transcription"]
    for turn in transcript
    if turn["speaker"].lower() == "executive"
])


Saving card_block_handling.json to card_block_handling (1).json


Define RAG-Based Policy Criteria

In [9]:
# Define RAG-grounded QA policy evaluation criteria
policy_criteria = [
    {"id": "verify_identity", "keywords": ["mobile number", "date of birth", "last four digits"], "weight": 0.20},
    {"id": "deny_unblock", "keywords": ["cannot unblock", "not allowed to unblock", "blocked permanently"], "weight": 0.20},
    {"id": "give_security_reason", "keywords": ["for security reasons", "due to security", "security policy"], "weight": 0.15},
    {"id": "reissue_card", "keywords": ["reissue", "replacement card", "new card issued"], "weight": 0.15},
    {"id": "mention_delivery_timeline", "keywords": ["5 to 7 working days", "5–7 days", "delivered within"], "weight": 0.10},
    {"id": "mention_tracking", "keywords": ["track", "tracking", "SMS confirmation", "mobile app"], "weight": 0.10},
    {"id": "empathy_politeness", "keywords": ["happy to help", "thank you", "assist", "please"], "weight": 0.10},
]


Scoring Logic

In [10]:
def check_criterion_met(transcript_text, keywords):
    transcript_lower = transcript_text.lower()
    return any(kw.lower() in transcript_lower for kw in keywords)

def compute_qa_score(transcript_text, policy_criteria):
    total_score = 0
    detailed_results = []
    for item in policy_criteria:
        met = check_criterion_met(transcript_text, item["keywords"])
        score = item["weight"] if met else 0
        total_score += score
        detailed_results.append({
            "criterion": item["id"],
            "met": met,
            "score": round(score, 2)
        })
    return round(total_score, 2), detailed_results


 Run QA Evaluation

In [11]:
qa_score, details = compute_qa_score(executive_transcript, policy_criteria)

print(f"\n✅ Final QA Score: {qa_score}/1.0\n")
print("📋 Detailed Breakdown:\n")
for d in details:
    status = "✅" if d["met"] else "❌"
    print(f"{status} {d['criterion']} → Score: {d['score']}")



✅ Final QA Score: 0.8/1.0

📋 Detailed Breakdown:

✅ verify_identity → Score: 0.2
❌ deny_unblock → Score: 0
✅ give_security_reason → Score: 0.15
✅ reissue_card → Score: 0.15
✅ mention_delivery_timeline → Score: 0.1
✅ mention_tracking → Score: 0.1
✅ empathy_politeness → Score: 0.1
